In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install msgpack
!{sys.executable} -m pip install mpmath
!{sys.executable} -m pip install powerlaw
import powerlaw
import numpy as np
import scipy.io as sio

Requirement already up-to-date: pip in /anaconda3/envs/avalanche/lib/python3.7/site-packages (19.1.1)


In [9]:
np.seterr(divide='ignore', invalid='ignore')
basedir = '/Users/harangju/Developer'
x = sio.loadmat(basedir + '/durs.mat')
dur_var_name = 'durs'
(nrows,ncols) = x[dur_var_name].shape
T = x['durs_max'][0][0]

In [10]:
f = [0]*nrows
for i in range(0,nrows):
#     print('MLE for ({}/{},{}/{})...'.format(i+1,nrows,j+1,ncols))
    print('MLE for {}/{}...'.format(i+1,nrows))
    d = x[dur_var_name][i][0]
    d = d[d<T]
    if len(d) <= 1:
        continue
    f[i] = powerlaw.Fit(d, discrete=True)

MLE for 1/25...
MLE for 2/25...
MLE for 3/25...


Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


MLE for 4/25...
MLE for 5/25...


Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


MLE for 6/25...
MLE for 7/25...


...

Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


MLE for 19/25...
MLE for 20/25...
MLE for 21/25...
MLE for 22/25...


Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


MLE for 23/25...
MLE for 24/25...
MLE for 25/25...


Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


In [11]:
p = np.zeros((nrows,3))
for i in range(0,nrows):
    for j in range(0,ncols):
        try:
            p[i] = [f[i].truncated_power_law.parameter1, \
                      f[i].truncated_power_law.parameter2, \
                      f[i].truncated_power_law.xmin]
        except:
            print("{} not available".format(i))
p

array([[2.62179051e+00, 2.09121077e-04, 4.00000000e+00],
       [2.43949810e+00, 5.69049029e-03, 3.20000000e+01],
       [2.16516806e+00, 9.14199272e-05, 5.00000000e+00],
       [2.12564036e+00, 3.81622922e-05, 4.00000000e+00],
       [2.34171585e+00, 1.28472755e-04, 7.00000000e+00],
       [2.21024438e+00, 2.85071510e-06, 4.00000000e+00],
       [2.14225383e+00, 1.77487717e-06, 3.00000000e+00],
       [1.88329142e+00, 1.20245081e-04, 1.10000000e+01],
       [1.94984108e+00, 1.89693678e-03, 7.00000000e+00],
       [3.88258115e+00, 1.06617142e-03, 2.79000000e+02],
       [2.07360979e+00, 2.76424697e-05, 8.00000000e+00],
       [1.78185705e+00, 5.39609970e-03, 1.48000000e+02],
       [1.73069697e+00, 1.97380752e-03, 3.00000000e+00],
       [1.92171845e+00, 1.97822230e-03, 5.00000000e+00],
       [2.45735269e+00, 1.06662387e-10, 2.00000000e+00],
       [1.81037973e+00, 1.69250364e-04, 4.00000000e+00],
       [2.05780936e+00, 6.83692785e-05, 7.00000000e+00],
       [1.84662102e+00, 3.46265

In [12]:
t_pl_exp = np.zeros((nrows,2))
t_pl_tpl = np.zeros((nrows,2))
t_tpl_exp = np.zeros((nrows,2))
for i in range(0,nrows):
    try:
        R, pval = f[i].distribution_compare('power_law', 'exponential', normalized_ratio = True)
        t_pl_exp[i] = [R, pval]
        R, pval = f[i].distribution_compare('power_law', 'truncated_power_law', normalized_ratio = True)
        t_pl_tpl[i] = [R, pval]
        R, pval = f[i].distribution_compare('truncated_power_law', 'exponential', normalized_ratio = True)
        t_tpl_exp[i] = [R, pval]
    except:
        print("{} not available".format(i))

Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions
Assuming nested distributions


In [13]:
sio.savemat('/Users/harangju/Developer/ft_pl.mat',
            {'ft_pl':p,'t_pl_exp':t_pl_exp,'t_pl_tpl':t_pl_tpl,'t_tpl_exp':t_tpl_exp})